In [3]:
import pandas as pd
import pymorphy2
import nltk
import re

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import datetime
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
from gensim.models import *
from gensim import corpora
from threading import Thread
from time import sleep

dir_hackathon = '.'
dir_data_in = f'{dir_hackathon}/Data/In'
dir_data_out = f'{dir_hackathon}/Data/Out'



Подготовка


Тесты

In [9]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = nltk.corpus.stopwords.words("russian")


def lemmatize(text_in):
    lemma = pymorphy2.MorphAnalyzer()
    text = " ".join([lemma.parse(word)[0].normal_form for word in text_in.split(' ')])
    tokens = word_tokenize(text, language="russian")
    for token in tokens:
        if token in stop_words:
            tokens.remove(token)
    text = " ".join(tokens)
    return text


def get_metrics(product_name, claster_words_precomputed, clasters_common_words, count_clasters, ): # Брать 2 вариант
  product_name = lemmatize(product_name)
  product_name_words = product_name.split(' ')
  list_name_words = pd.unique(product_name_words)

  list_k = []
  for id_claster in range(count_clasters): # Внутри кластера суммировать или умножать - подумать    
    # print(range(count_clasters))
    claster_words = claster_words_precomputed[id_claster]['claster_words']
    claster_count_words = claster_words_precomputed[id_claster]['claster_count_words']
    claster_len = clasters_common_words.iloc[id_claster]['Общее количество слов'] # !!!
    # Tfidf - поискать наименование метрики - ??? 

    # print(f'Claster id: {str(id_claster)}')
    k = 0
    for word in list_name_words:  
      word_name_count = product_name_words.count(word)

      if word in claster_words:
        id_word = claster_words.index(word) 
        word_claster_count = claster_count_words[id_word] # !!!

        k += word_claster_count / (claster_len * len(list_name_words) / len(claster_words)) # !!!!! Подумать, как правильно !!!
        
    list_k.append(k)

  return list_k

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rexarrior\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rexarrior\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
import json
with open(f'{dir_data_out}/data_to_str_vectors.json', "rt", encoding="utf8") as f:
    json_str = f.read()
claster_words_precomputed = json.loads(json_str)
pd_clasters_common_words = pd.read_excel(f'{dir_data_out}/clasters_common_words.xlsx')

product_names = pd.read_excel(f'{dir_data_in}/product_group_names.xlsx')
print(product_names.head(5))
count_clasters = 2067
product_names_dict = {}
for i in range(product_names.shape[0]):
    product_names_dict[product_names.iloc[i]['Группа продукции_ordinal']] = product_names.iloc[i]['Группа продукции'] 


   Unnamed: 0                                   Группа продукции  \
0       56825  Автоматические установки жидкостного и пенного...   
1          28  Автоматические устройства управления бытовыми ...   
2       21178  Автоматические устройства управления бытовыми ...   
3       27932                                     Автопогрузчики   
4       20374  Адаптивные системы переднего освещения; Жгуты ...   

   Группа продукции_ordinal  
0                       0.0  
1                       1.0  
2                       2.0  
3                       3.0  
4                       4.0  


In [14]:

xl = pd.ExcelFile(f'{dir_data_in}/Аполлон_Тесты.xlsx')

print(xl.sheet_names)  # see all sheet names


['тест1', 'тест2']


Task 1

In [15]:
test_dataset = xl.parse('тест1')
test_dataset.head(2)

,Номер продукции,Общее наименование продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Наличие ошибки
0,Продукция S 1697,"Кондиционеры бытовые (Сплит-системы),",9032890000,О безопасности низковольтного оборудования; Эл...,Автоматические устройства управления бытовыми ...,NaN
1,Продукция S 440,"Кондиционеры бытовые (Сплит-системы),",8537109800,О безопасности низковольтного оборудования; Эл...,Автоматические устройства управления бытовыми ...,NaN


In [ ]:
for i in range(test_dataset.shape[0]):
    print(f"process {i} row")
    row = test_dataset.iloc[i]
    list_k = get_metrics(row['Общее наименование продукции'], claster_words_precomputed,  pd_clasters_common_words)
    max_coef = np.max(list_k)
    choosen_cluster  = list_k.index(max_coef)
    choosen_cluster_name =  product_names_dict[choosen_cluster]
    row['Наличие ошибки'] = 
    pass